# Hotspot

In [8]:
import requests

# Your API key
api_key = "3g5voge8rcai"

# Region code for the entire United States
region_code = "US"

# API endpoint
url = f"https://api.ebird.org/v2/ref/hotspot/{region_code}"

# Query parameters
params = {
    "back": 30,   # last 30 days (max allowed)
    "fmt": "json" # return JSON format
}

# Headers including your API key
headers = {
    "X-eBirdApiToken": api_key
}

# Make the request
response = requests.get(url, params=params, headers=headers)

# Check response
if response.status_code == 200:
    hotspots = response.json()
    print(f"Number of hotspots returned: {len(hotspots)}\n")
    print("Sample hotspot:")
    print(hotspots[0])  # print first hotspot entry
else:
    print("Error:", response.status_code, response.text)
    

Number of hotspots returned: 52541

Sample hotspot:
{'locId': 'L10913808', 'locName': '"The Maze" Trail', 'countryCode': 'US', 'subnational1Code': 'US-UT', 'subnational2Code': 'US-UT-053', 'lat': 37.294524, 'lng': -113.692871, 'latestObsDt': '2025-11-15 14:02', 'numSpeciesAllTime': 20}


In [9]:
import pandas as pd

# Convert JSON list to DataFrame
df = pd.DataFrame(hotspots)

# Save to CSV
df.to_csv("us_hotspots_last30days.csv", index=False)

print("CSV saved as us_hotspots_last30days.csv")

CSV saved as us_hotspots_last30days.csv


In [16]:
import pandas as pd
import json
import os 

base_dir = "/Users/xingyechen/Downloads/DataEngineering_FinalProject/map"

hotspot_file = os.path.join(base_dir, "us_hotspots_last30days.csv")
df_hot = pd.read_csv(hotspot_file)

# Extract state code (US-CA → CA)
df_hot["state"] = df_hot["subnational1Code"].str.split("-").str[1]

# === OPTION A: HOTSPOT COUNT PER STATE ===
hotspot_count = df_hot.groupby("state").size().reset_index(name="hotspot_count")

# === OPTION B: SPECIES RICHNESS PER STATE (use max species per hotspot in that state) ===
species_richness = df_hot.groupby("state")["numSpeciesAllTime"].max().reset_index(name="species_richness")

# === MERGE TO SINGLE DATAFRAME ===
state_stats = hotspot_count.merge(species_richness, on="state")

print(state_stats.head())

  state  hotspot_count  species_richness
0    AK            404               325
1    AL            368               343
2    AR            407               301
3    AZ           1231               339
4    CA           6669               445


## Store into Database

In [17]:
import requests
import pandas as pd
import sqlite3
import os

# === 1. Call eBird Hotspot API ===
api_key = "3g5voge8rcai"
region_code = "US"

url = f"https://api.ebird.org/v2/ref/hotspot/{region_code}"

params = {"back": 30, "fmt": "json"}
headers = {"X-eBirdApiToken": api_key}

response = requests.get(url, params=params, headers=headers)

if response.status_code != 200:
    raise Exception(f"API Error {response.status_code}: {response.text}")

hotspots = response.json()
print("Hotspots returned:", len(hotspots))

# === 2. Convert JSON to DataFrame ===
df = pd.DataFrame(hotspots)

# Extract 2-letter state code (US-NY → NY)
df["state"] = df["subnational1Code"].str.split("-").str[1]

# === 3. Compute state-level stats ===
hotspot_count = df.groupby("state").size().reset_index(name="hotspot_count")
species_richness = df.groupby("state")["numSpeciesAllTime"].max().reset_index(name="species_richness")

state_stats = hotspot_count.merge(species_richness, on="state")

print("\nState Stats Preview:")
print(state_stats.head())

# === 4. SAVE TO NORMALIZED DATABASE ===

db_path = os.path.join(os.getcwd(), "ebird_hotspots.db")
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# --- Create normalized schema ---
cur.executescript("""
DROP TABLE IF EXISTS hotspots;
DROP TABLE IF EXISTS states;
DROP TABLE IF EXISTS state_stats;

CREATE TABLE states (
    state_code TEXT PRIMARY KEY,
    state_name TEXT
);

CREATE TABLE hotspots (
    hotspot_id TEXT PRIMARY KEY,
    locName TEXT,
    state_code TEXT,
    county_code TEXT,
    lat REAL,
    lng REAL,
    latestObsDt TEXT,
    species_richness INTEGER,
    FOREIGN KEY (state_code) REFERENCES states(state_code)
);

CREATE TABLE state_stats (
    state_code TEXT PRIMARY KEY,
    hotspot_count INTEGER,
    species_richness INTEGER,
    FOREIGN KEY (state_code) REFERENCES states(state_code)
);
""")

# === Insert STATES table ===
states_unique = pd.DataFrame({"state_code": df["state"].unique()})
states_unique["state_name"] = None  # optional for now

states_unique.to_sql("states", conn, if_exists="append", index=False)

# === Insert HOTSPOTS table ===
hotspots_table = pd.DataFrame({
    "hotspot_id": df["locId"],
    "locName": df["locName"],
    "state_code": df["state"],
    "county_code": df["subnational2Code"],
    "lat": df["lat"],
    "lng": df["lng"],
    "latestObsDt": df["latestObsDt"],
    "species_richness": df["numSpeciesAllTime"],
})

hotspots_table.to_sql("hotspots", conn, if_exists="append", index=False)

# === Insert STATE_STATS table ===
state_stats.rename(columns={"state": "state_code"}, inplace=True)
state_stats.to_sql("state_stats", conn, if_exists="append", index=False)

conn.close()

print(f"\nDatabase saved → {db_path}")
print("Tables created: states, hotspots, state_stats")

Hotspots returned: 52548

State Stats Preview:
  state  hotspot_count  species_richness
0    AK            394               325
1    AL            373               343
2    AR            414               301
3    AZ           1220               339
4    CA           6555               445

Database saved → /Users/xingyechen/Downloads/DataEngineering_FinalProject/ebird_hotspots.db
Tables created: states, hotspots, state_stats


In [18]:
import numpy as np

# Add a log column to reduce skew
state_stats['log_hotspot'] = np.log1p(state_stats['hotspot_count'])  # log(1 + x)
state_stats['log_richness'] = np.log1p(state_stats['species_richness'])

In [19]:
# Load Geo Info
geo_path = os.path.join(base_dir, "us-states.json")
with open(geo_path, "r") as f:
    us_geo = json.load(f)

In [ ]:
state_stats.head(10)

,state_code,hotspot_count,species_richness,log_hotspot,log_richness
0,AK,394,325,5.978886,5.786897
1,AL,373,343,5.924256,5.840642
2,AR,414,301,6.028279,5.710427
3,AZ,1220,339,7.107425,5.828946
4,CA,6555,445,8.788136,6.100319
5,CO,1558,353,7.351800,5.869297
6,CT,838,327,6.732211,5.793014
7,DC,88,266,4.488636,5.587249
8,DE,207,334,5.337538,5.814131
9,FL,2827,354,7.947325,5.872118


In [20]:
# Example mapping 2-letter codes to FIPS
state_fips = {
    'AL': '01','AK':'02','AZ':'04','AR':'05','CA':'06', 
    'CO':'08','CT':'09','DE':'10','FL':'12','GA':'13',
    'HI':'15','ID':'16','IL':'17','IN':'18','IA':'19',
    'KS':'20','KY':'21','LA':'22','ME':'23','MD':'24',
    'MA':'25','MI':'26','MN':'27','MS':'28','MO':'29',
    'MT':'30','NE':'31','NV':'32','NH':'33','NJ':'34',
    'NM':'35','NY':'36','NC':'37','ND':'38','OH':'39',
    'OK':'40','OR':'41','PA':'42','RI':'44','SC':'45',
    'SD':'46','TN':'47','TX':'48','UT':'49','VT':'50',
    'VA':'51','WA':'53','WV':'54','WI':'55','WY':'56'
}

state_stats['fips'] = state_stats['state_code'].map(state_fips)

In [21]:
columns=["fips", "log_hotspot"]
key_on="feature.id"

In [22]:
# Check the top-level keys
print(us_geo.keys())  
# Should show: dict_keys(['type', 'features'])

# Inspect the first feature
first_feature = us_geo['features'][0]
print(first_feature)

dict_keys(['type', 'features'])
{'type': 'Feature', 'id': '01', 'properties': {'name': 'Alabama', 'density': 94.65}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}


In [29]:
import folium
import pandas as pd
import os
import json

# === CONFIG ===
base_dir = "/Users/xingyechen/Downloads/DataEngineering_FinalProject/map"
species_files = {
    "acowoo": os.path.join(base_dir, "acowoo.csv"),
    "ameavo": os.path.join(base_dir, "ameavo.csv"),
    "amhgul1": os.path.join(base_dir, "amhgul1.csv"),
    "amwpel": os.path.join(base_dir, "amwpel.csv"),
    "annhum": os.path.join(base_dir, "annhum.csv"),
}

species_colors = {
    "acowoo": "red",
    "ameavo": "blue",
    "amhgul1": "green",
    "amwpel": "orange",
    "annhum": "purple"
}

# Load US states GeoJSON with state codes as "id"
geo_path = os.path.join(base_dir, "us-states.json")  # download from PublicaMundi
with open(geo_path) as f:
    us_geo = json.load(f)

# === BASE MAP ===
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# === ADD SPECIES LAYERS ===
for code, path in species_files.items():
    fg = folium.FeatureGroup(name=f"Species: {code}", show=False)
    if os.path.exists(path):
        df = pd.read_csv(path)
        for _, row in df.iterrows():
            lat = row.get("LATITUDE") or row.get("lat")
            lon = row.get("LONGITUDE") or row.get("lng")
            if pd.isna(lat) or pd.isna(lon):
                continue

            popup = folium.Popup(
                f"<b>{row.get('COMMON NAME', row.get('comName', ''))}</b><br>"
                f"<i>{row.get('SCIENTIFIC NAME', row.get('sciName', ''))}</i><br>"
                f"Location: {row.get('LOCATION NAME', row.get('locName', ''))}<br>"
                f"Count: {row.get('OBSERVATION COUNT', row.get('howMany', ''))}<br>"
                f"Date: {row.get('OBSERVATION DATE', row.get('obsDt', ''))}",
                max_width=250
            )

            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color=species_colors.get(code, "black"),
                fill=True,
                fill_opacity=0.8,
                popup=popup
            ).add_to(fg)
    fg.add_to(m)

# Hotspot choropleth with log scale
folium.Choropleth(
    geo_data=us_geo,
    data=state_stats,
    columns=["fips", "log_hotspot"],   # use the FIPS column
    key_on="feature.id",               # match GeoJSON feature id
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Hotspot Count (log scale)",
    name="Hotspot Count",
    show=False
).add_to(m)

# Species richness choropleth with log scale
folium.Choropleth(
    geo_data=us_geo,
    data=state_stats,
    columns=["fips", "log_richness"],  # use the FIPS column
    key_on="feature.id",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Species Richness (log scale)",
    name="Species Richness",
    show=False
).add_to(m)

# === LAYER CONTROL ===
folium.LayerControl(collapsed=False).add_to(m)

# === LIMIT TO ONE SPECIES LAYER AT A TIME (JS) ===
js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var checkboxes = document.querySelectorAll('.leaflet-control-layers-selector[type="checkbox"]');
    checkboxes.forEach(function(cb) {
        cb.addEventListener('change', function() {
            if (this.nextSibling.textContent.trim().startsWith('Species:')) {
                checkboxes.forEach(function(other) {
                    if (other !== cb && other.checked && other.nextSibling.textContent.trim().startsWith('Species:')) {
                        other.click();
                    }
                });
            }
        });
    });
});
</script>
"""

output_path = os.path.join(base_dir, "ebird_species_hotspot_map.html")
m.save(output_path)

# Inject JS at the end
with open(output_path, "a") as f:
    f.write(js)

print(f"✅ Map saved to: {output_path}")
print("→ Open it in your browser. You can toggle species or state metrics.")

✅ Map saved to: /Users/xingyechen/Downloads/DataEngineering_FinalProject/map/ebird_species_hotspot_map.html
→ Open it in your browser. You can toggle species or state metrics.


# Density

In [24]:
# Helper: convert lat/lng to state using bounding boxes
def latlon_to_state(lat, lon):
    for state in us.states.STATES:
        if state.bounding_box is None:
            continue
        min_lon, min_lat, max_lon, max_lat = state.bounding_box
        if (min_lat <= lat <= max_lat) and (min_lon <= lon <= max_lon):
            return state.abbr
    return None


In [27]:
import geopandas as gpd
from shapely.geometry import Point

# Load US states polygons
shp_path = os.path.join(base_dir, "shapefiles", "cb_2022_us_state_20m.shp")
states_gdf = gpd.read_file(shp_path)[["STUSPS", "geometry"]]  # STUSPS = 2-letter code

def latlon_to_state(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return None

    point = Point(lon, lat)  # shapely uses (x=lon, y=lat)

    for _, row in states_gdf.iterrows():
        if row["geometry"].contains(point):
            return row["STUSPS"]
    return None

In [30]:
import folium
import pandas as pd
import os
import json
import us  

# === CONFIG ===
base_dir = "/Users/xingyechen/Downloads/DataEngineering_FinalProject/map"
species_files = {
    "acowoo": os.path.join(base_dir, "acowoo.csv"),
    "ameavo": os.path.join(base_dir, "ameavo.csv"),
    "amhgul1": os.path.join(base_dir, "amhgul1.csv"),
    "amwpel": os.path.join(base_dir, "amwpel.csv"),
    "annhum": os.path.join(base_dir, "annhum.csv"),
}

species_colors = {
    "acowoo": "red",
    "ameavo": "blue",
    "amhgul1": "green",
    "amwpel": "orange",
    "annhum": "purple"
}

# Load US states GeoJSON with state codes as "id"
geo_path = os.path.join(base_dir, "us-states.json")  # download from PublicaMundi
with open(geo_path) as f:
    us_geo = json.load(f)

# === BASE MAP ===
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# === ADD SPECIES LAYERS ===
for code, path in species_files.items():
    fg = folium.FeatureGroup(name=f"Species: {code}", show=False)
    if os.path.exists(path):
        df = pd.read_csv(path)
        for _, row in df.iterrows():
            lat = row.get("LATITUDE") or row.get("lat")
            lon = row.get("LONGITUDE") or row.get("lng")
            if pd.isna(lat) or pd.isna(lon):
                continue

            popup = folium.Popup(
                f"<b>{row.get('COMMON NAME', row.get('comName', ''))}</b><br>"
                f"<i>{row.get('SCIENTIFIC NAME', row.get('sciName', ''))}</i><br>"
                f"Location: {row.get('LOCATION NAME', row.get('locName', ''))}<br>"
                f"Count: {row.get('OBSERVATION COUNT', row.get('howMany', ''))}<br>"
                f"Date: {row.get('OBSERVATION DATE', row.get('obsDt', ''))}",
                max_width=250
            )

            folium.CircleMarker(
                location=[lat, lon],
                radius=5,
                color=species_colors.get(code, "black"),
                fill=True,
                fill_opacity=0.8,
                popup=popup
            ).add_to(fg)
    fg.add_to(m)

# Hotspot choropleth with log scale
folium.Choropleth(
    geo_data=us_geo,
    data=state_stats,
    columns=["fips", "log_hotspot"],   # use the FIPS column
    key_on="feature.id",               # match GeoJSON feature id
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Hotspot Count (log scale)",
    name="Hotspot Count",
    show=False
).add_to(m)

# Species richness choropleth with log scale
folium.Choropleth(
    geo_data=us_geo,
    data=state_stats,
    columns=["fips", "log_richness"],  # use the FIPS column
    key_on="feature.id",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Species Richness (log scale)",
    name="Species Richness",
    show=False
).add_to(m)

# === SPECIES DENSITY LAYERS ===
for code, path in species_files.items():

    if not os.path.exists(path):
        continue

    df = pd.read_csv(path)

    # Find lat/lon columns
    lat = df.get("LATITUDE", df.get("lat"))
    lon = df.get("LONGITUDE", df.get("lng"))

    df["state"] = df.apply(
        lambda r: latlon_to_state(
            r.get("LATITUDE") or r.get("lat"),
            r.get("LONGITUDE") or r.get("lng")
        ),
        axis=1
    )

    df = df.dropna(subset=["state"])

    # Count observations per state
    density = df.groupby("state").size().reset_index(name="density")

    # Map to FIPS codes
    density["fips"] = density["state"].map(state_fips)

    # Add choropleth
    folium.Choropleth(
        geo_data=us_geo,
        data=density,
        columns=["fips", "density"],
        key_on="feature.id",
        fill_color="PuBuGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"{code} Density (observations per state)",
        name=f"Species Density: {code}",
        show=False
    ).add_to(m)

# === LAYER CONTROL ===
folium.LayerControl(collapsed=False).add_to(m)

# === LIMIT TO ONE SPECIES LAYER AT A TIME (JS) ===
js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var checkboxes = document.querySelectorAll('.leaflet-control-layers-selector[type="checkbox"]');
    checkboxes.forEach(function(cb) {
        cb.addEventListener('change', function() {
            if (this.nextSibling.textContent.trim().startsWith('Species:')) {
                checkboxes.forEach(function(other) {
                    if (other !== cb && other.checked && other.nextSibling.textContent.trim().startsWith('Species:')) {
                        other.click();
                    }
                });
            }
        });
    });
});
</script>
"""

output_path = os.path.join(base_dir, "ebird_species_density.html")
m.save(output_path)

# Inject JS at the end
with open(output_path, "a") as f:
    f.write(js)

print(f"✅ Map saved to: {output_path}")
print("→ Open it in your browser. You can toggle species or state metrics.")

✅ Map saved to: /Users/xingyechen/Downloads/DataEngineering_FinalProject/map/ebird_species_density.html
→ Open it in your browser. You can toggle species or state metrics.
